## Resize Database

In [ ]:
import os
import cv2 
import glob

Database_path = '/media/nhattuong/Data/TranseNet/TransNet_Database'
Databse_resize = '/media/nhattuong/Data/TranseNet/Database_resized_batch3/'

img_paths = sorted(glob.glob(f'{Database_path}/KeyFramesC02_*/*/*.jpg'))

for img_path in img_paths:
    # if img_path.split('/')[-3] == 'KeyFramesC02_V01':
    #     print('Skip')
    #     continue

    os.makedirs(Databse_resize + '/'.join(img_path.split('/')[-3:-1]), exist_ok=True)

    des_path = Databse_resize + '/'.join(img_path.split('/')[-3:])

    # if os.path.exists(des_path):
    #     continue 

    print("des_path: ", des_path)

    img = cv2.imread(img_path)
    img = cv2.resize(img, (224,224))

    cv2.imwrite(des_path, img)

In [ ]:
import os
import cv2 
from glob import glob
import pandas as pd
from tqdm import tqdm

list_csv_paths = '/media/nhattuong/Data/Dataset_BTC/keyframe_p_batch3'
list_frame_paths = '/media/nhattuong/Data/TranseNet/TransNet_Database'

lst_csv = glob(f'{list_csv_paths}/*.csv')
print("lst_csv: ", lst_csv)
lst_csv.sort()
dct_names = {}

for csv_path in tqdm(lst_csv):
    df = pd.read_csv(csv_path,header = None)
    for i in df.index:
        row = df.loc[i]
        video_id = csv_path.split('/')[-1][:-4]
        key = f'{video_id}/{row[0]}'
        value = f'{video_id}/{row[1]:06}.jpg' 
        dct_names[key] = value

prev_keyframe = ''
for key, value in tqdm(dct_names.items()):
    keyframe = f'KeyFrames{key.split("/")[0][:-2]}' # KeyFramesC00_V00
    frame_src_path = f'{list_frame_paths}/{keyframe}/{keyframe}/{key}'
    frame_dst_path = f'{list_frame_paths}/{keyframe}/{keyframe}/{value}'

    print(frame_src_path, frame_dst_path)

    if frame_src_path == frame_dst_path or not os.path.exists(frame_src_path):
        continue

    if prev_keyframe != keyframe:
        lst_frame_in_video = os.listdir('/'.join(frame_src_path.split('/')[:-1]))
        prev_keyframe = keyframe

    # if frame_dst_path.split('/')[-1] in lst_frame_in_video:
    #     os.remove(frame_src_path)
    # else:
    #     os.rename(frame_src_path, frame_dst_path)

## Create Files for MutilModel 

In [ ]:
batch1: 103108
batch2: 88185
batch3: 231239
All: 422532

In [ ]:
json_path = '/media/nhattuong/Data/AIC_2022/dict_support_model'
root_database = '/media/nhattuong/Data/Video-Text-Retrieval/Database'
shot_frames_path = '/media/nhattuong/Data/Dataset/scenes_txt'

count = 0
debug = 0
infos = []
des_path = os.path.join(json_path, "dict_support_model_batch.json")
keyframe_paths = sorted(glob.glob(f'{root_database}/KeyFramesC0*')) #_V01

for kf in keyframe_paths:
  video_paths = sorted(glob.glob(f"{kf}/*"))

  for video_path in video_paths:
    image_paths = sorted(glob.glob(f'{video_path}/*.jpg'))

    ###### Get all id keyframes from video_path ######
    id_keyframes = np.array([int(id.split('/')[-1].replace('.jpg', '')) for id in image_paths])

    ###### Get scenes from video_path ######
    video_info = video_path.split('/')[-1]
    with open(f'{shot_frames_path}/{video_info}.txt', 'r') as f:
      lst_range_shotes = f.readlines()
    lst_range_shotes = np.array([re.sub('\[|\]', '', line).strip().split(' ') for line in lst_range_shotes]).astype(np.uint32)

    for im_path in image_paths:
      im_path = 'Database/' + '/'.join(im_path.split('/')[-3:])
      id = int(im_path.split('/')[-1].replace('.jpg', ''))
      
      i = 0
      flag=0
      for range_shot in lst_range_shotes:
        i+=1
        first, end = range_shot

        if first <= id <= end:
          break
        
        if i == len(lst_range_shotes):
          flag=1
      
      if flag == 1:
        print(f"Skip: {im_path}")
        print(first, end)
        continue

      ##### Get List Shot ID #####
      lst_shot = id_keyframes[np.where((id_keyframes>=first) & (id_keyframes<=end))]
      lst_shot = [f"{i:0>6d}" for i in lst_shot]

      ##### Len > 7 #####
      if len(lst_shot) > 7:
        # print(lst_shot)
        # print(im_path)

        count_split = int(np.ceil(len(lst_shot)/7))
        c = len(lst_shot)//count_split

        # print(count_split, c)

        for i in range(count_split):
          if i + 1 == count_split:
            lst_split = lst_shot[i*c:]
          else:
            lst_split = lst_shot[i*c:(i+1)*c]

          # print(lst_split)
          # print(len(lst_split)//2)
          # print(lst_split.index(f"{id:0>6d}"))

          try:
            if len(lst_split)//2 == lst_split.index(f"{id:0>6d}"):
              ##### Merge All Info #####
              info = {
                      "image_path": im_path,
                      "list_shot_id": lst_split
                      }
                          
              infos.append(info) 
              count += 1
          except:
            pass

      else:
        if len(lst_shot)//2 == lst_shot.index(f"{id:0>6d}"):
          ##### Merge All Info #####
          info = {
                  "image_path": im_path,
                  "list_shot_id": lst_shot
                  }
                      
          infos.append(info) 
          count += 1

id2img_fps = dict(enumerate(infos))

with open(des_path, 'w') as f:
  f.write(json.dumps(id2img_fps))

print(f'Saved {des_path}')
print(f"Number of Index: {count}")